# Design


- 부품/모듈 조합을 설계하고 genbank 및 fasta 파일 생성
- 설계는 엑셀로 수행하고 해당 파일을 입력으로 genbank 및 fasta 파일을 생성해주는 파이썬 코드 활용
- 부품 정보는 모든 사용자 공유, 어셈블리는 각 사용자별, 프로젝트별 별도 관리/생성
- 설계는 다음 네 단계로 이루어지며 각각 엑셀파일로 설계를 수행함
  - 부품 정보 관리 (PartDB-kribb.xlsx)
  - Golden gate assembly용 부품 단위 설계 (part_preparation.xlsx)
  - 부품 어셈블리 기반 모듈 설계 (part_assembly.xlsx)
  - 모듈 어셈블리 기반 회로 설계 (module_assembly.xlsx)

## 부품 정보 관리

- PartDB-kribb.xlsx 파일에 저장
- 모든 사용자들 공유 
- 해당 excel file에는 순수 서열정보만 저장하고 
- 해당 부품이 enzyme site나 overhang과 같이 임의의 벡터에 삽입되어 저장된 경우는 해당 정보를 genbank 파일로 저장
- 부품 실물은 duplex 형태로 준비 
- CDS 실물은 임의의 공통 vector에 BsaI site와 overhang 붙인 상태로 저장

## Golden gate assembly용 부품 단위 설계 
- 사용자별로 별도의 프로젝트 생성하고 프로젝트별로 어셈블리 genbank 파일 관리 가능
- Golden gate 수행 가능한 상태의 part 단위의 정보 (genbank 파일) 및 실물 (DNA stock) 준비 과정
- 부품 실물은 본 시뮬레이션과는 다르게 duplex 형태로 저장되나 정보 (genbank file) 구성은 동일

### 부품 단위 설계 (엑셀파일)
- part_prparation.xlsx 파일로 설계 (아래 그림)
- 해당 part 서열에 enzyme site와 overhang 등을 붙여서 golden gate assembly 수행 되도록 설계
- 설계된 파일은 pUC19 vector에 저장됨을 가정함 (genbank/addgene-plasmid-50005-sequence-222046.gbk) 
  - 그러나 실제로 각 부품들은 overhang과 enzyme site가 붙어서 duplex 형태로 저장되므로 (주문 약 10~30만원, 1000배 희석, 0.1 microl/회 사용) 이 후 시뮬레이션 과정은 실제로 수행되지 않음. 본 시뮬레이션은 genbank 파일을 작성하기 위한 과정임. 
  - 준비된 정보는 [id 이름]-insert.gb으로 genbank_part 디렉토리에 저장
- 최종 genbank 파일 작성을 위해서 insert를 pUC19 vector에 클로닝하는 (Gibson assembly) 시뮬레이션을 수행  

![alt text](images/part_preparation.png){width=500px}

### Insert 준비
- Gibson으로 (simulation) 위 부품 insert를 vector에 넣기 위한 insert genbank 파일 만드는 작업 
- 위에서 작성된 excel file로 insert genbank 파일 생성
- genbank 파일에는 부품 서열에 BsaI, Overhang 만 붙여서 위치정보 저장 
- genbank_part 디렉토리에 저장
- gbfile_generator.py 파일에 구현

In [102]:
import part_preparation as pprep
from importlib import reload
reload(pprep)
import os

project_dir = "genbank_haseong0621"
part_preparation_file = os.path.join(project_dir, "part_preparation.xlsx")
design_file = os.path.join(project_dir, "part_assembly.xlsx")
path_design_file = os.path.join(project_dir, "module_assembly.xlsx")
part_db_file = "PartDB-kribb-hs.xlsx"
vector_gbfile = "genbank/addgene-plasmid-50005-sequence-222046.gb"

pprep.part_insert_prep_goldengate(project_dir, part_preparation_file, part_db_file)

Process completed. GenBank files have been created in the folder of /home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/part/insert


### Gibson assembly for part preparation
- Gibson을 활용한 insert - vector에 클로닝 (gbfile 얻기 위한 시뮬레이션)
- pUC19의 MCS 서열과 교환 삽입
  - 교환 삽입은 pydna에서 지원하지 않아 enzyme cut 후 pcr 수행하는 방법으로 진행
- Gibson 위한 각 insert용 primer 서열 자동 생성 및 저장 수행 
- pydna의 assembly후 insert의 feature 위치가 적절히 업데이트 되지 않는 문제로 관련 fix 포함

In [103]:
import part_preparation as pprep
import os
from importlib import reload
reload(pprep)

pprep.part_withvector_prep_gibson(project_dir, vector_gbfile)

Process completed. GenBank files have been created in the folder of /home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/part/withvector


## 부품 어셈블리 기반 모듈 설계 

- 엑셀파일에 어셈블리 할 부품 조합 디자인  (part_assembly.xlsx)
- part_assembly.py 에 구현
- golden gate ready 부품들로 한정 (앞서 만든 genbank_part 디렉토리의 부품들로 한정됨)
- 동일 id에 속한 부품들이 조합되어 하나의 linear DNA 형성
- 조립된 하나의 모듈은 벡터에 저장되며 genbank 파일로 정보 저장
- genbank 파일은 엑셀파일에 지정된 id를 파일이름으로 genbank_module 디렉토리에 저장 

![alt text](images/partassembly.png)

In [104]:
import part_assembly as pasm
import pandas as pd 
from importlib import reload
reload(pasm)

combinations = pasm.get_part_combinations(design_file)
# display(pd.DataFrame(combinations))

allcomb = pasm.get_all_possible_combinations(project_dir, combinations)
print("len:", len(allcomb))

filtered_combinations = pasm.filtering_combinations(allcomb)
print("len:", len(filtered_combinations))


len: 27
len: 27


- 위 조합은 part_assembly.xlsx 엑셀파일에 명시된 부품들의 조합과 같음
- 그러나 디자인 파일에서 보듯 부품들의 이름만으로 설계를 수행하는 점 때문에 더 많은 조합이 있을 수 있음
  - 예를 들어 BBa_J23100 부품으로 디자인 했으나 gb 파일에는 앞 뒤 오버행이 붙는 형태의 부품이 저장되고 O1-BBa_J23100-O2-withvector.gb 파일과 O2-BBa_J23100-O3-withvector.gb 파일이 같은 BBa_J23100 부품으로 취급될 수 있음.
  - 정확히 하기 위해서는 각 부품마다 오버행을 어떻게 붙이느냐에 따라 다른 부품으로 취급해야 함
  - 그러나 디자인 편의를 위해 부품의 이름만으로 수행하고 
  - 이는 사용자가 원하는 조합 외에도 다른 여러 조합이 발생할 가능성이 있음을 의미 
- 이러한 문제 해결을 위해 연결부위에 공통 오버행을 갖는 부품들로만 구성되어 있는 조합을 먼저 필터링하고 이 후 어셈블리 시뮬레이션으로 실제 어셈블리 예측 검증하고 최종 gb 파일과 primer 리스트 반환

### 설계된 조합별로 Linear DNA 조립 Goldengate Assembly
- 앞서 만들어진 부품 조합들을 goldengate assembly로 linear dna 만들기
- genbank file 저장

In [105]:
import part_assembly as pasm
import pandas as pd 
from importlib import reload
reload(pasm)
from pydna.all import read

## goldengate assembly with the filtered combinations
## 
module_linear = pasm.part_assembly_goldengate(project_dir, filtered_combinations)


/home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/module/inserts/VA8-BBa_J23106-BBa_B0034-IspA-L2U3H03-VA2.gb

### Gibson assembly for building a module
- Assume pUC19 is used for the vector

In [106]:
import part_assembly as pasm
import pandas as pd 
from importlib import reload
reload(pasm)

module_withvector_list = pasm.build_module_withvector_gibson(project_dir, vector_gbfile, module_linear)

/home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/module/withvector/VA8-BBa_J23106-BBa_B0034-IspA-L2U3H03-VA2_withvector.gb

## 모듈 어셈블리 기반 대사경로 설계
- 각 모듈들의 조합에 대해서 gibson assembly 이용한 linear DNA 만들기
- module_assembly.xlsx 파일로 디자인
- 아래 이미지와 같이 ID를 임의로 넣고 원하는 조합의 유전자(CDS)를 명시함 
- 해당 유전자들이 포함된 모든 조합에 대해서 linear DNA 먼저 생성 
- pathway/insert/[modulename] 에 gbfile 생성
- module_assembly.py에 구현


![alt text](images/moduleassemblydesign.png)

In [107]:
import module_assembly as masm
from importlib import reload
reload(masm)

assembly_linear_dict = masm.module_combinatorial_gibson_assembly(project_dir, path_design_file, vector_gbfile)


/home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/pathway/insert/MM2/MM2_0080.gb

- 주어진 vector에 gibson으로 삽입한 상태의 circular DNA 제작
- pathway/withvector/MMxx에 genbank 파일 저장

In [128]:
import module_assembly as masm
from importlib import reload
reload(masm)

assembly_circular_dict = masm.module_comb_withvector_gibson_assembly(project_dir, vector_gbfile, assembly_linear_dict)


/home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/pathway/withvector/MM2/MM2_0080_withvector.gb

In [129]:
display(len(assembly_linear_dict['MM1']))
display(len(assembly_linear_dict['MM2']))

108

81

## 대사경로 조합
- 위 두 개 이상의 모듈이 조합된 대사경로들에 대해서 서로 모든 조합으로 조립하는 과정

In [167]:
import module_assembly as masm
from importlib import reload
reload(masm)

all_pathway_combination_list = masm.pathway_comb_withvector_gibson_assembly(project_dir, path_design_file, vector_gbfile, assembly_linear_dict)



MM3_0010_withvector.gb


/home/haseong/dev/mvaopt/labnote/002_part_preparation/genbank_haseong0621/pathway/withvector/MM3/MM3_0010_withvector.gb

In [172]:
all_pathway_combination_list[0].list_features()

Ft#,Label or Note,Dir,Sta,End,Len,type,orf?
0,L:M13 Forward,<--,0,18,18,primer_bind,no
1,L:M13/pUC Forwar,<--,9,32,23,primer_bind,no
2,L:pRS-marker,<--,225,245,20,primer_bind,no
3,L:pGEX 3',-->,344,367,23,primer_bind,no
4,L:pBRforEco,<--,404,423,19,primer_bind,no
5,L:AmpR promoter,-->,490,595,105,promoter,no
6,L:AmpR,-->,595,1456,861,CDS,yes
7,L:Amp-R,<--,813,833,20,primer_bind,no
8,L:ori,-->,1626,2215,589,rep_origin,no
9,L:pBR322ori-F,-->,2115,2135,20,primer_bind,no
